## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,New Norfolk,AU,2020-11-09 04:10:46,-42.78,147.06,78.01,28,37,3.00,scattered clouds
1,1,Dikson,RU,2020-11-09 04:10:46,73.51,80.55,-1.41,91,95,14.65,overcast clouds
2,2,Palmer,US,2020-11-09 04:10:46,42.16,-72.33,48.00,100,1,0.60,clear sky
3,3,Rikitea,PF,2020-11-09 04:10:46,-23.12,-134.97,73.09,78,89,15.66,light rain
4,4,Arraial Do Cabo,BR,2020-11-09 04:10:46,-22.97,-42.03,73.90,93,70,8.12,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
31,31,Padang,ID,2020-11-09 04:10:50,-0.95,100.35,84.49,66,75,4.83,light rain
36,36,Pangai,TO,2020-11-09 04:10:51,-19.80,-174.35,80.60,74,20,8.05,few clouds
37,37,Butaritari,KI,2020-11-09 04:10:51,3.07,172.79,82.11,65,1,8.99,clear sky
42,42,Trinidad,BO,2020-11-09 04:10:51,-14.83,-64.90,82.24,50,1,1.95,clear sky
46,46,Balabac,PH,2020-11-09 04:10:52,7.99,117.06,83.97,73,79,12.91,broken clouds
54,54,San Policarpo,PH,2020-11-09 04:10:53,12.18,125.51,85.30,73,83,8.30,light rain
69,69,Kapaa,US,2020-11-09 04:10:55,22.08,-159.32,80.60,69,75,13.87,broken clouds
73,73,Esperance,AU,2020-11-09 04:10:56,-33.87,121.90,91.40,27,40,18.34,scattered clouds
74,74,Kahului,US,2020-11-09 04:10:56,20.89,-156.47,82.40,58,1,19.46,clear sky
76,76,Hambantota,LK,2020-11-09 04:10:56,6.12,81.12,82.40,88,20,3.36,few clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()


City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
31,Padang,ID,84.49,light rain,-0.95,100.35,
36,Pangai,TO,80.60,few clouds,-19.80,-174.35,
37,Butaritari,KI,82.11,clear sky,3.07,172.79,
42,Trinidad,BO,82.24,clear sky,-14.83,-64.90,
46,Balabac,PH,83.97,broken clouds,7.99,117.06,
54,San Policarpo,PH,85.30,light rain,12.18,125.51,
69,Kapaa,US,80.60,broken clouds,22.08,-159.32,
73,Esperance,AU,91.40,scattered clouds,-33.87,121.90,
74,Kahului,US,82.40,clear sky,20.89,-156.47,
76,Hambantota,LK,82.40,few clouds,6.12,81.12,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...")

Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...
Hotel not found...


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
31,Padang,ID,84.49,light rain,-0.95,100.35,Mercure Padang
36,Pangai,TO,80.60,few clouds,-19.80,-174.35,Ha'apai Beach Resort
37,Butaritari,KI,82.11,clear sky,3.07,172.79,Isles Sunset Lodge
42,Trinidad,BO,82.24,clear sky,-14.83,-64.90,Hotel Piesta
46,Balabac,PH,83.97,broken clouds,7.99,117.06,Sing and Swing Lodge


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Max Temp} *F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))